# Tutorial 5: A Detailed Look at Agents - Custom LLMs, Structured Output, and Workflows

Welcome to the fifth tutorial in the Aurite Agents series! Building on what you learned in the previous tutorials, we'll now explore more advanced features that give you greater control over your AI agents.

**In this tutorial, you will learn:**

1. **Custom LLM Configurations** - How to create and use your own LLM settings beyond the defaults
2. **Structured Output** - How to enforce JSON schemas so agents return data in exactly the format you need
3. **Simple Workflows** - How to chain multiple agents together to create more complex behaviors

By the end of this tutorial, you'll have built a multi-agent workflow that demonstrates all three concepts working together.

> 📋 **Prerequisites**
> This tutorial assumes you've completed the previous tutorials and are familiar with basic agent creation and execution.

## ✅ 1. Setup and Installation

Let's start with the same setup as the previous tutorials.

In [1]:
%pip install aurite==0.3.14

### Configure Your API Key

As before, we need to set up your OpenAI API key for this tutorial. Follow the same process from the previous tutorials:

- **Google Colab Users:** Use the **Secrets** tab (🔑) to create a secret named `OPENAI_API_KEY`
- **Local IDE Users:** Create a `.env` file with `OPENAI_API_KEY=sk-your-key-here`

In [2]:
import os

# Load API key safely
try:
    from google.colab import userdata #type: ignore
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
except ImportError:
    from dotenv import load_dotenv
    load_dotenv()
except Exception as e:
    print(f"Could not load API key. Please ensure it's set up correctly. Error: {e}")

if os.getenv("OPENAI_API_KEY"):
    print("✅ OpenAI API Key is set and ready!")
else:
    print("❌ OpenAI API Key not found. Please follow the setup steps above.")

In [3]:
from IPython.display import display, Markdown

def display_agent_response(agent_name: str, query: str, response: str):
  """Formats and displays the agent's response in a structured Markdown block."""

  output = f"""
  <div style=\"border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);\">
    <div style=\"background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;\">
      <h3 style=\"margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;\">
        <span style=\"margin-right: 8px;\">🤖</span>
        Agent Response: <code style=\"background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;\">{agent_name}</code>
      </h3>
    </div>
    <div style=\"padding: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Your Query:</strong>
      </p>
      <p style=\"background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;\">
        <em>\"{query}\"</em>
      </p>
      <hr style=\"border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Result:</strong>
      </p>
      <div style=\"background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;\">
        {response}
      </div>
    </div>
  </div>
  """
  display(Markdown(output))

## 🔧 2. Custom LLM Configurations

In Tutorial 1, we used the default LLM settings (GPT-4 Turbo). But what if you want different settings? Maybe you want:

- A faster, cheaper model for simple tasks
- Different temperature settings for more creative or more predictable responses
- Custom token limits
- A different system prompt that applies to all agents using this LLM

This is where **LLM Configurations** come in.

In [4]:
# Initialize Aurite
from aurite import Aurite, AgentConfig, LLMConfig

aurite = Aurite()
await aurite.initialize()

print("✅ Aurite initialized!")

### Creating a Custom LLM Configuration

Let's create two different LLM configurations:
1. A **fast, cheap** configuration using GPT-4o-mini
2. A **creative** configuration with higher temperature

In [5]:
# Create a fast, economical LLM configuration
fast_llm = LLMConfig(
    llm_id="fast_gpt",
    provider="openai",
    model_name="gpt-4o-mini",
    temperature=0.2,  # Lower temperature = more predictable
    max_tokens=1024,  # Fewer tokens = faster + cheaper
    default_system_prompt="You are a helpful, concise assistant. Keep responses brief and to the point."
)

# Create a creative LLM configuration
creative_llm = LLMConfig(
    llm_id="creative_gpt",
    provider="openai",
    model_name="gpt-4o",
    temperature=0.9,  # Higher temperature = more creative
    max_tokens=2048,
    default_system_prompt="You are a creative, imaginative assistant. Feel free to be expressive and think outside the box."
)

# Register both LLM configurations with Aurite
await aurite.register_llm_config(fast_llm)
await aurite.register_llm_config(creative_llm)

print("✅ Custom LLM configurations registered!")

### Using Custom LLM Configurations in Agents

Now let's create agents that use our custom LLM configurations:

In [6]:
# Create an agent using the fast LLM
fast_agent = AgentConfig(
    name="Fast Summarizer",
    llm_config_id="fast_gpt",  # Reference our custom LLM
    system_prompt="Summarize the user's input in exactly one sentence."
)

# Create an agent using the creative LLM
creative_agent = AgentConfig(
    name="Creative Storyteller",
    llm_config_id="creative_gpt",  # Reference our custom LLM
    system_prompt="Turn the user's input into a short, imaginative story."
)

# Register both agents
await aurite.register_agent(fast_agent)
await aurite.register_agent(creative_agent)

print("✅ Agents with custom LLMs registered!")

### Testing the Different LLM Behaviors

Let's see how the same input produces different outputs based on our LLM configurations:

In [7]:
user_input = "A robot discovers an old library filled with forgotten books. \
It starts reading them and finds a book that changes its understanding of the world. \
It decides to share this knowledge with other robots, sparking a revolution of thought and creativity among them."

# Test the fast agent
fast_result = await aurite.run_agent(
    agent_name="Fast Summarizer",
    user_message=user_input
)

display_agent_response(
    agent_name="Fast Summarizer (GPT-4o-mini, temp=0.2)",
    query=user_input,
    response=fast_result.primary_text
)

In [8]:
# Test the creative agent
creative_result = await aurite.run_agent(
    agent_name="Creative Storyteller",
    user_message=user_input
)

display_agent_response(
    agent_name="Creative Storyteller (GPT-4o, temp=0.9)",
    query=user_input,
    response=creative_result.primary_text
)

Notice how the **Fast Summarizer** gives you a concise, predictable response, while the **Creative Storyteller** produces a more imaginative and varied output. This demonstrates how LLM configurations let you tune the behavior for different use cases.

## 📊 3. Structured Output with JSON Schema

Sometimes you don't want free-form text from your agent—you want **structured data** in a specific format. This is where `config_validation_schema` comes in.

With a JSON schema, you can force your agent to return data in exactly the structure you need, making it easy to use the output in your application code.

### Creating an Agent with Structured Output

Let's create an agent that analyzes text and returns structured data about it:

In [9]:
# Define a JSON schema for our structured output
text_analysis_schema = {
    "type": "object",
    "properties": {
        "sentiment": {
            "type": "string",
            "enum": ["positive", "negative", "neutral"],
            "description": "The overall sentiment of the text"
        },
        "key_topics": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Main topics or themes identified in the text"
        },
        "word_count": {
            "type": "integer",
            "description": "Approximate number of words in the text"
        },
        "summary": {
            "type": "string",
            "description": "A brief summary of the text content"
        }
    },
    "required": ["sentiment", "key_topics", "word_count", "summary"]
}

# Create an agent that returns structured analysis
analysis_agent = AgentConfig(
    name="Text Analyzer",
    llm_config_id="fast_gpt",  # Use our fast LLM for this structured task
    system_prompt="You are a text analysis expert. Analyze the user's text and return your analysis as a JSON object matching the provided schema. CRITICAL: Your response MUST be a single JSON object matching the schema provided. Do not include any other text or explanations in your response - just the JSON object.",
    config_validation_schema=text_analysis_schema
)

await aurite.register_agent(analysis_agent)
print("✅ Text Analyzer agent with structured output registered!")

### Testing Structured Output

Now let's test our structured output agent:

In [10]:
text_to_analyze = """
I absolutely love the new AI features in this software!
The machine learning capabilities are incredibly impressive and have streamlined our workflow significantly.
Our team productivity has increased by 30% since we started using these tools.
The natural language processing component is particularly noteworthy.
"""

analysis_result = await aurite.run_agent(
    agent_name="Text Analyzer",
    user_message=f"Please analyze this text: {text_to_analyze}"
)

display_agent_response(
    agent_name="Text Analyzer (Structured Output)",
    query="Analyze the provided text",
    response=analysis_result.primary_text
)

### Working with the Structured Data

Since the output is guaranteed to be valid JSON, we can easily parse it and use it in our code:

In [11]:
import json

# Parse the JSON response
try:
    analysis_data = json.loads(analysis_result.primary_text)

    print("📊 Parsed Analysis Results:")
    print(f"   Sentiment: {analysis_data['sentiment']}")
    print(f"   Word Count: {analysis_data['word_count']}")
    print(f"   Key Topics: {', '.join(analysis_data['key_topics'])}")
    print(f"   Summary: {analysis_data['summary']}")

except json.JSONDecodeError:
    print("❌ Failed to parse JSON response")

## 🔗 4. Building Simple Workflows

Now let's combine everything we've learned to create a **workflow**—a sequence of agents that work together to accomplish a more complex task.

We'll build a **Content Processing Workflow** that:
1. **Step 1:** Takes raw text and creates a structured analysis
2. **Step 2:** Takes that analysis and generates recommendations based on it

### Creating the Workflow Agents

First, let's create our two specialized agents:

In [12]:
# Agent 1: Content Analyzer (with structured output)
content_analyzer_schema = {
    "type": "object",
    "properties": {
        "content_type": {
            "type": "string",
            "enum": ["marketing", "technical", "educational", "news", "social", "other"],
            "description": "The type/category of content"
        },
        "target_audience": {
            "type": "string",
            "enum": ["general", "technical", "business", "academic", "youth"],
            "description": "The intended audience for this content"
        },
        "complexity_level": {
            "type": "integer",
            "minimum": 1,
            "maximum": 5,
            "description": "Content complexity on a scale of 1-5"
        },
        "key_themes": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Main themes or topics in the content"
        }
    },
    "required": ["content_type", "target_audience", "complexity_level", "key_themes"]
}

content_analyzer = AgentConfig(
    name="Content Analyzer",
    llm_config_id="fast_gpt",
    system_prompt="You are a content analysis expert. Analyze the provided text and categorize it according to the schema. CRITICAL: Your response MUST be a single JSON object matching the schema provided. Do not include any other text or explanations.",
    config_validation_schema=content_analyzer_schema
)

# Agent 2: Recommendation Generator (takes structured input, gives text output)
recommendation_generator = AgentConfig(
    name="Recommendation Generator",
    llm_config_id="creative_gpt",
    system_prompt="You are a content strategy expert. You will receive a JSON analysis of some content. Based on this analysis, provide 3-5 specific, actionable recommendations for how to improve or better utilize this content. Format your recommendations as a numbered list."
)

# Register both agents
await aurite.register_agent(content_analyzer)
await aurite.register_agent(recommendation_generator)

print("✅ Workflow agents registered!")

### Creating and Registering the Workflow

Now let's create a `SimpleWorkflow` that chains these agents together:

In [13]:
from aurite import WorkflowConfig

# Create a workflow that chains our two agents
content_workflow = WorkflowConfig(
    name="Content Processing Workflow",
    steps=["Content Analyzer", "Recommendation Generator"],
    description="Analyzes content and generates improvement recommendations"
)

# Register the workflow
await aurite.register_workflow(content_workflow)

print("✅ Content Processing Workflow registered!")

### Running the Workflow

Now let's test our complete workflow. Keep in mind that your initial input here is the message that you are sending to the first agent in the workflow. This agent is not aware of the second agent (unless you explain it in the system prompt)

In [14]:
# Sample content to process
sample_content = """
Introducing Neural Networks: A Beginner's Guide

Neural networks are computing systems inspired by biological neural networks.
They consist of interconnected nodes (neurons) that process information through weighted connections.
These systems excel at pattern recognition, classification, and prediction tasks.
Modern applications include image recognition, natural language processing, and autonomous vehicles.
While the mathematics can be complex, the core concept is intuitive:
networks learn by adjusting connection weights based on training data.
This tutorial series will guide you through building your first neural network using Python.
"""

# Run the workflow
workflow_result = await aurite.run_workflow(
    workflow_name="Content Processing Workflow",
    initial_input=f"Please process this content: {sample_content}"
)

print("🔄 Workflow completed! Here is the final message:")
# The .final_message property is a convenient way to get the primary text from the last step
print(workflow_result.final_message)

### Displaying Workflow Results

Let's examine the output from each step of our workflow:

In [15]:
# Display results from each step
from aurite.components.agents.agent_models import AgentExecutionResult

for i, step_result in enumerate(workflow_result.step_results, 1):
    print(f"\n📋 Step {i}: {step_result.step_name} ({step_result.step_type})")
    print("-" * 50)

    # The result of each step is a Pydantic model, which we can inspect
    if isinstance(step_result.result, AgentExecutionResult):
        # This was an agent step. We can display its output.
        if step_result.step_name == "Content Analyzer":
            try:
                parsed_analysis = json.loads(step_result.result.primary_text) # Parse the JSON output
                print("📊 Structured Analysis:")
                for key, value in parsed_analysis.items(): # Display each key-value pair
                    print(f"   {key}: {value}")
            except (json.JSONDecodeError, TypeError):
                print(f"Raw output: {step_result.result.primary_text}")
        else:
            print(f"💡 Recommendations:\n{step_result.result.primary_text}")

print("\n" + "="*60)

## 🎉 Congratulations!

You've successfully completed Tutorial 5! You now know how to:

### ✅ What You've Learned:

1. **Custom LLM Configurations**
   - Create LLM configs with different models, temperatures, and token limits
   - Use `llm_config_id` to assign specific LLMs to agents
   - Tune behavior for different use cases (fast vs. creative)

2. **Structured Output**
   - Define JSON schemas using `config_validation_schema`
   - Force agents to return data in exactly the format you need
   - Parse and use structured data in your applications

3. **Simple Workflows**
   - Chain multiple agents together in sequence
   - Pass output from one agent as input to the next
   - Build complex behaviors from simple, specialized agents

### 🚀 Next Steps:

In the next tutorial, we'll explore:
- **Custom Tool Servers** - Creating your own tools that agents can use
- **Advanced Workflows** - More complex agent coordination patterns
- **Real-world Integration** - Connecting agents to external APIs and services

### 💡 Key Takeaway:

The power of Aurite Agents comes from **composition**—combining simple, well-defined components (LLMs, agents, workflows) to build sophisticated AI applications. Each component has a single responsibility, making your system modular, testable, and maintainable.